# This notebook serves as walkthrough for planning an experiment for creation through the OT2.
### The following modules are used and should be in the same directory as this notebook: 
* **CreateSamples** is responsible for calculating sample information, which includes component weight fractions and stock volumes
* **OT2Commands** is responsible for setting up information to be interpretted and executed by opentrons.
* **OT2Graphing** contains graphing tools to help visualize and explore parameter spaces.

In [ ]:
import CreateSamples
import OT2Commands as ALH
import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api

# Would not load
import importlib # for reloading packages
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
importlib.reload(CreateSamples)
importlib.reload(ALH)
importlib.reload(ographing)

## Step 1: Set up the experiment dictionary.
* The first step to planning an experiment is to load the experiment variables and inputs from a csv file. Every variable should have an input with an acceptable datatype. At the moment this step is done by opening a CSV file in Excel, where the first column is the name of the variable and the adjacent column is the variable value. The default delimitter is (,). 
    * Reading directly as csv is fine but it requires you have all data values in a string so then we can use ast.literal_eval to unpack this and the appropiate datatypes. This forces you \to put marks ('') around each variable value when planning the experiment. NOTE: You still need to place marks around anything inside a dtype i.e components inside a list.
        * To remove this dependency we can build our own interpreters for our specfic cases such as to not use ast.literal_evals default unpacking.
    * Loading from excel can be done in a similar manner but is avoided due not having xlrd or openpyxl depdenency native to python, and the opentrons being limited in the packages we can add/update. Hence we default to a CSV.
* **The experiment dictionary consist of keys being the variable name and the value being the variables value.**

In [ ]:
path = r"C:\Users\Edwin\Desktop\OT2-DOE\Plan and Prepare\Testing Plans\02_25_21_Ouzo_Search.csv"
experiment_csv_dict = CreateSamples.get_experiment_plan(path) 

## Step 1a Optional: Load custom labware dictionary (Remote Testing)
* Provide the path to the directory holding all custom labware. This directory should have custom labware .json files you have previously made and tested, read more here: https://support.opentrons.com/en/articles/3136504-creating-custom-labware-definitions
* The reason we provide this is when working on a device that is not connected to the OT2's Jupyter notebook there is no way to natively use custom labware. So we create a dictionary of custom labware so we can later load into our protocol to primarily simualte and test protocols for execution later once connnected to the OT2's notebook.
    * When using custom labware on the OT2's notebook it pulls from a folder labeled "labware", which is something built into the Opentrons hardware. It has not been tested if the custom labware dictionary will superceed this directory of custom labware if used on the OT2.

In [ ]:
labware_dir_path = r"C:\Users\Edwin\Desktop\OT2-DOE\Plan and Prepare\Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)

## Step 2: Select and Create Sampling Space
* Create sampling space depending on the units of concentration and method of sampling. All information is pulled from the experimental dictionary made in Step 1.
    * Currently the only sampling method available are simple lattice and random based sampling. There are two potential ways to create samples in a system of n components which currently utilzie the linspaces of concentration. 
        * *Remember the linspace of concentration refers to [minimum concentration, maximum concentration, concentration step size]*
    * **Case 1 (Completing case):** Specify all but one (in this case the last) component's concentrations, which with the addition of exposing the unity_filter = True, would calculate the the remaining concentraiton values using the information of the last index of all component related variables (i.e. names). This is only meant for units that require unity like volf, wtf, and molf. 
    * **Case 2 (Non-completing case):** Specify all concentration linspaces, not applying any unit based filters, meant for all other non interdepedent units like molarity and mg/mL.
* Other things to take into consideration: All units must be the same for unity based, but not for non-unity units.

In [ ]:
wtf_sample_canidates = CreateSamples.generate_candidate_lattice_concentrations(experiment_csv_dict, unity_filter=True)
# o
wtf_sample_canidates
# something interesting you can do here is after making these wtf samples you can now apply some random selection

## Step 3: Calculate Volumes of Stocks
* From the concentration values calculated in Step 2, we use those along with stock concentration information to calculate the volume of required for each sample.

* This is where things get less "*general*" each case depending on the number of stocks, common components (i.e. component A in both stock A and B) and other requirements will typically require its own function. Luckily given the commmonality of using data frames this should be quite simple. 
* Currently the only function to calculate volumes in centered around the Ouzo emulsion systems. This system consist of 3 stock with the solvent being ethanol and two pure stocks of ethanol and water. 

* Ideally the way volumes should calculated is simply by calculating "*essential information*" given the concentration of component and the systems overall mass or volume. Using this essential information and the concentration unit of the stock, it should call the appropiate function to calcualte the volume. Many issue could arise such as having a molarity and providing a mass so would need to make sure these cases are sorted and reported back


In [ ]:
volume_sample_canidates = CreateSamples.calculate_ouzo_volumes_from_wtf(wtf_sample_canidates, experiment_csv_dict)
volume_sample_canidates

## Step 4: Create Complete Component Cocentration and Volume Dataframe and Apply Filters

In [ ]:
complete_df = CreateSamples.combine_df(wtf_sample_canidates, volume_sample_canidates) # unfiltered
# complete_df = pd.concat([complete_df], ignore_index=True)
# Step 3: Apply filters through df based logic, currently 4 filters exist (volume, total, general min and general max)

# First filter for pipette volume constraints, optional Volume Restriction to select certain components for filter application ("stock" must be in column name)
complete_df_f1 = CreateSamples.pipette_volume_restriction_df(complete_df, 30, 1000, experiment_csv_dict['Volume Restriction']) # last argument is optional

# Second filter for overall total volume_restriction, call max destination well volume ("Total Sample Volume" must be in column name)
# max_dest_well_volume = ALH.find_max_dest_volume_labware(experiment_csv_dict, custom_labware_dict)
complete_df_f2 = CreateSamples.total_volume_restriction_df(complete_df_f1,800)

#Thrid filter for any general max or min filtering you would like
final_complete_df = complete_df_f2#CreateSamples.general_max_restriction(complete_df_f2, 360, 'pfh-ethanol-stock uL')
final_complete_df

## Step 4a (Optional): Visual

In [ ]:
%matplotlib qt
ographing.xy_scatter_df_compare(complete_df, final_complete_df, 'ethanol wtf', 'pfh wtf')
# ographing.xy_scatter_df(final_complete_df, 'ethanol wtf', 'pfh molf')

## Step 5: Finalize and Call Seperate Concentration and Volume Dataframes

In [ ]:
final_wtf_df = CreateSamples.isolate_common_column(final_complete_df, 'wtf')
final_volume_df = CreateSamples.isolate_common_column(final_complete_df, 'stock')
final_volume_df

## Step 6 (Optional): Calculate Stock Prep Information

In [ ]:
chem_database_path = r"Chemical Database.xlsx"
stock_prep_df = CreateSamples.calculate_stock_prep_df(experiment_csv_dict, final_volume_df, chem_database_path, buffer_pct=25)
# pd.set_option('display.float_format', lambda x: '%.2e' % x)
stock_prep_df
# so acutally have it print with concentration, just in case you go iver 

## Step 7: Set up Ranges for Stocks

In [ ]:
protocol = simulate.get_protocol_api('2.0', extra_labware=custom_labware_dict)
max_vol = 18000 
stock_ranges = ALH.stock_well_ranges(final_volume_df, max_vol) # set up volumes orders
stock_ranges # make sure there is an assertion when number of ranges exceeds number sof well

In [ ]:
# Step 8: Simulate/Execute

In [ ]:
# please for the love of a higher power add easier way to call distribute function, and modularize the way we call commands,
# it should be easy since you already have the set up of pipettes and labware seperate from volume handinling commands, jsut need ot make more neater

In [ ]:
importlib.reload(ALH)


In [ ]:
protocol = simulate.get_protocol_api('2.0', extra_labware=custom_labware_dict)
loaded_dict = ALH.loading_labware(protocol, experiment_csv_dict) # the protocol above has been modified globally!
info = ALH.pipette_stock_volumes(protocol, loaded_dict, final_volume_df, stock_ranges)

In [ ]:
## Step 9: Uploaded to Google Drive

In [ ]:
CreateSamples.create_csv(r"C:\Users\Edwin\Desktop\test", info['info concat'], final_wtf_df.values, experiment_csv_dict)
df = pd.read_csv(r"C:\Users\Edwin\Desktop\test")
